## Different models for  sentiment analysis on the  IMDb Dataset 

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [0]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df = pd.read_csv('/content/drive/My Drive/Masterarbeit_Elvis/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [0]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
print(X_train.shape)
for i in range(0,5):
    print(X_train[i])
    print("")

(25000,)
I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.

Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises f

In [0]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [0]:
# summarize size
print("Training data: ")
print(X.shape)
print(y.shape)

Training data: 
(50000,)
(50000,)


In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


tokenizer_obj = Tokenizer()
total_reviews = X_train + X_test
tokenizer_obj.fit_on_texts(total_reviews) 

# pad sequences
max_length = max([len(s.split()) for s in total_reviews])

# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens =  tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='pre')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='pre')

In [0]:
print(vocab_size)

125602


In [0]:
print(X_train_pad.shape)
print(X_train_pad[0])#just numers now

(25000, 2678)
[   0    0    0 ...  158   22 1717]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Conv1D, GlobalAveragePooling1D
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 50

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(Conv1D(filters=50,kernel_size=(40)))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Build model...
Summary of the built model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2678, 50)          6280100   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2639, 50)          100050    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 6,380,201
Trainable params: 6,380,201
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2678, 50)          6280100   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2639, 50)          100050    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 6,380,201
Trainable params: 6,380,201
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print(max_length)
print(vocab_size)

2678
125602


In [0]:
np.random.seed(22)
idx=np.random.choice(range(0,25000),replace=False,size=20000)

In [0]:
X_train_pad =X_train_pad[idx] 
y_train=y_train[idx] 
X_test_pad=X_test_pad[idx] 
y_test=y_test[idx] 

In [0]:
X_train_pad[0]

array([   0,    0,    0, ...,   48,   22, 1371], dtype=int32)

In [0]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=64, epochs=10, validation_data=(X_test_pad, y_test), verbose=1)

Train...
Train on 20000 samples, validate on 20000 samples
Epoch 1/10
20000/20000 [==============================] - 12s 606us/step - loss: 0.6136 - acc: 0.6306 - val_loss: 0.4122 - val_acc: 0.8154
Epoch 2/10
20000/20000 [==============================] - 12s 579us/step - loss: 0.2949 - acc: 0.8794 - val_loss: 0.4000 - val_acc: 0.8256
Epoch 3/10
20000/20000 [==============================] - 12s 586us/step - loss: 0.1958 - acc: 0.9212 - val_loss: 0.3317 - val_acc: 0.8790
Epoch 4/10
20000/20000 [==============================] - 12s 580us/step - loss: 0.1325 - acc: 0.9490 - val_loss: 0.4081 - val_acc: 0.8619
Epoch 5/10
20000/20000 [==============================] - 11s 572us/step - loss: 0.0970 - acc: 0.9648 - val_loss: 0.4095 - val_acc: 0.8736
Epoch 6/10
20000/20000 [==============================] - 11s 567us/step - loss: 0.0704 - acc: 0.9729 - val_loss: 0.5243 - val_acc: 0.8527
Epoch 7/10
20000/20000 [==============================] - 11s 564us/step - loss: 0.0550 - acc: 0.9800 - val

In [0]:
# summarize history 
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

NameError: ignored

In [0]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

Testing...
5000/5000 [==============================] - 26s 5ms/step
Test score: 0.6712235076904297
Test accuracy: 0.6188
Accuracy: 61.88%


In [0]:
#Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

#predict
model.predict(x=test_samples_tokens_pad)

array([[0.5373639 ],
       [0.5381622 ],
       [0.53790957],
       [0.53790396],
       [0.53692925],
       [0.5380995 ],
       [0.5381007 ],
       [0.53709775]], dtype=float32)

In [0]:
#let us check how the model predicts
classes = model.predict(X_test_pad[:10], batch_size=128)
for i in range (0,10):
    if(classes[i] > 0.5 and y_test[i] == 1 or (classes[i] <= 0.5 and y_test[i] == 0)):
        print( classes[i], y_test[i], " Right prdiction")
    else :
        print( classes[i], y_test[i], " Wrong prdiction")

[0.1360948] 1  Wrong prdiction
[0.29261437] 1  Wrong prdiction
[0.9970799] 1  Right prdiction
[0.87384164] 1  Right prdiction
[0.99331176] 1  Right prdiction
[0.97243744] 1  Right prdiction
[0.85155106] 1  Right prdiction
[0.7417877] 1  Right prdiction
[0.9908635] 1  Right prdiction
[0.9956833] 1  Right prdiction


In [0]:
from keras.datasets import imdb
from keras.models import Sequential
from tensorflow.python.keras.preprocessing import sequence
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('Build model...')

model = Sequential()
model.add(Embedding(top_words, 100, input_length=max_words))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print('Train...')

model.fit(X_train, y_train, batch_size=128, epochs=25, validation_data=(X_test, y_test), verbose=2)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/25
 - 513s - loss: 0.1798 - acc: 0.9324 - val_loss: 0.4092 - val_acc: 0.8448
Epoch 2/25
 - 556s - loss: 0.1890 - acc: 0.9283 - val_loss: 0.4166 - val_acc: 0.8473
Epoch 3/25
 - 946s - loss: 0.1736 - acc: 0.9345 - val_loss: 0.4416 - val_acc: 0.8446
Epoch 4/25
 - 651s - loss: 0.1599 - acc: 0.9400 - val_loss: 0.4562 - val_acc: 0.8464
Epoch 5/25
 - 688s - loss: 0.1635 - acc: 0.9390 - val_loss: 0.4382 - val_acc: 0.8432
Epoch 6/25
 - 1021s - loss: 0.1507 - acc: 0.9440 - val_loss: 0.4438 - val_acc: 0.8478
Epoch 7/25
 - 518s - loss: 0.1392 - acc: 0.9476 - val_loss: 0.4582 - val_acc: 0.8507
Epoch 8/25
 - 516s - loss: 0.1573 - acc: 0.9391 - val_loss: 0.4841 - val_acc: 0.8454
Epoch 9/25
 - 785s - loss: 0.1377 - acc: 0.9500 - val_loss: 0.4811 - val_acc: 0.8469
Epoch 10/25
 - 1003s - loss: 0.1353 - acc: 0.9508 - val_loss: 0.4742 - val_acc: 0.8494
Epoch 11/25
 - 630s - loss: 0.1753 - acc: 0.9315 - val_loss: 0.4758 - val_acc: 0.8385
Ep

In [0]:
score, acc = model.evaluate(X_test, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)
print("Accuracy: %.2f%%" % (acc*100))

25000/25000 [==============================] - 694s 28ms/step
Test score: 0.5993069805335999
Test accuracy: 0.839160000038147
Accuracy: 83.92%


The time to train a GRU is less than LSTM network.